In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report


In [2]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to the first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def preprocess_text_lemma(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]
    return ' '.join(words)




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harish-4072\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harish-4072\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\harish-4072\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\harish-4072\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\harish-4072\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [3]:
stemmer = PorterStemmer()
def stemming(text):
    words = word_tokenize(text)
    return ' '.join([stemmer.stem(word) for word in words])

In [4]:
train_df = pd.read_csv(r"D:\Kaggle\disaster tweets\nlp-getting-started\train.csv")
test_df = pd.read_csv(r"D:\Kaggle\disaster tweets\nlp-getting-started\test.csv")

In [5]:
date_pattern = r'\b(\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{4}[-/]\d{1,2}[-/]\d{1,2}|(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},?\s\d{4})\b'
time_pattern = r'\b((0?[1-9]|1[0-2]):[0-5]\d\s?(AM|PM)|([01]\d|2[0-3]):[0-5]\d(:[0-5]\d)?)\b'

def preprocess_text(text):
    text = re.sub(r'bin laden', 'Binladen', text, flags=re.IGNORECASE)
    text = re.sub(r"http\S+|www\S+|https\S+", 'http', text, flags=re.MULTILINE)  
    #text = re.sub(r'\@\w+|\#','', text)  
    text = re.sub(r'#(\w+)', r'\1', text)
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b(?<!breaking)news\b|\b(?<!breaking)\w*news\w*\b', 'news', text)
    return text

# train_df['text'] = train_df['location'].fillna('') + ' ' + train_df['text'].fillna('')
# test_df['text'] = test_df['location'].fillna('') + ' ' + test_df['text'].fillna('')
# train_df['text'] = train_df['keyword'].fillna('') + ' ' + train_df['text'].fillna('')
# test_df['text'] = test_df['keyword'].fillna('') + ' ' + test_df['text'].fillna('')
train_df['text'] = train_df['text'].apply(lambda x: re.sub(date_pattern, 'DATETIME', x))
train_df['text'] = train_df['text'].apply(lambda x: re.sub(time_pattern, 'DATETIME', x))
test_df['text'] = test_df['text'].apply(lambda x: re.sub(date_pattern, 'DATETIME', x))
test_df['text'] = test_df['text'].apply(lambda x: re.sub(time_pattern, 'DATETIME', x))
train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)
train_df['text'] = train_df['text'].apply(preprocess_text_lemma)
test_df['text'] = test_df['text'].apply(preprocess_text_lemma)
train_df['text'] = train_df['text'].apply(stemming)
test_df['text'] = test_df['text'].apply(stemming)

# train_df['url'] = train_df['text'].str.contains(r'http|https', regex=True)
# test_df['url'] = test_df['text'].str.contains(r'http|https', regex=True)
# train_df['contains_country'] = train_df['text'].str.contains(r'\b(israel|afghan|iran|iraq|lebanon|yemen|palestine)\b', regex=True, case=False)
# test_df['contains_country'] = train_df['text'].str.contains(r'\b(israel|afghan|iran|iraq|lebanon|yemen|palestine)\b', regex=True, case=False)

# train_df['isNews'] = train_df['text'].str.contains(r'news|News|Breakingnews|BreakingNews|breakingnews', regex=True)
# test_df['isNews'] = test_df['text'].str.contains(r'news|News|Breakingnews|BreakingNews|breakingnews', regex=True)


In [6]:
train_df.head(), train_df.size,test_df.size, train_df.isnull().sum(),test_df.isnull().sum()

(   id keyword location                                               text  \
 0   1     NaN      NaN  our deed be the reason of thi earthquak may al...   
 1   4     NaN      NaN               forest fire near la rong sask canada   
 2   5     NaN      NaN  all resid ask to shelter in place be be notifi...   
 3   6     NaN      NaN     peopl receiv wildfir evacu order in california   
 4   7     NaN      NaN  just get sent thi photo from rubi alaska a smo...   
 
    target  
 0       1  
 1       1  
 2       1  
 3       1  
 4       1  ,
 38065,
 13052,
 id             0
 keyword       61
 location    2533
 text           0
 target         0
 dtype: int64,
 id             0
 keyword       26
 location    1105
 text           0
 dtype: int64)

In [7]:
train_df_id = train_df['id']
test_df_id = test_df['id']
X = train_df['text']
y = train_df['target']
X_test = test_df['text']


In [8]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(X).toarray()  # Convert to array for PyTorch
y = y.values
X_test = vectorizer.transform(X_test).toarray()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor)


train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=128,shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128)

In [9]:


from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [10]:
import requests
from pathlib import Path 

if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

helper_functions.py already exists, skipping download


In [11]:
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
class TweetDisasterModel(nn.Module):
    def __init__(self,input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=input_shape,out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,out_features=output_shape)
        )

    def forward(self,X):
        return self.layer_stack(X)

In [13]:
model = TweetDisasterModel(X_train.shape[1],64,1)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [14]:
def accuracy_fn(y_pred,y_true):
    correct = torch.eq(y_true, (torch.sigmoid(y_pred)>=0.5).float()).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc




In [15]:
import torch


def train_mode(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, accuracy_fn, optimizer: torch.optim.Optimizer):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for batch, (X, y) in enumerate(data_loader):                
        y_pred = model(X)
        # loss = loss_fn(y,y_pred.squeeze())
        # train_loss += loss.item()
        # train_accuracy += accuracy_fn(y, y_pred.squeeze())
        loss = loss_fn(y_pred.squeeze(), y)
        train_loss += loss.item()
        train_accuracy += accuracy_fn(y_pred.squeeze(), y)
        #print(y_pred.shape,y_pred,y,train_accuracy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(data_loader.dataset)} samples")
    train_loss /= len(data_loader)
    train_accuracy /= len(data_loader)
    return train_loss, train_accuracy

def test_mode(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module, accuracy_fn, optimizer: torch.optim.Optimizer):
    model.eval()
    test_loss = 0
    test_accuracy = 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(data_loader):
            test_pred = model(X)
            test_loss += loss_fn(test_pred.squeeze(), y).item()
            test_accuracy += accuracy_fn(test_pred.squeeze(), y)
            # test_pred = model(X)
            # test_loss += loss_fn(y, test_pred.squeeze()).item()
            # test_accuracy += accuracy_fn(y, test_pred.squeeze())
            if batch % 400 == 0:
                print(f"Looked at {batch * len(X)}/{len(data_loader.dataset)} samples")
        test_loss /= len(data_loader)
        test_accuracy /= len(data_loader)
    return test_loss, test_accuracy


In [16]:
from tqdm.auto import tqdm
torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 100
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_loss, train_accuracy = train_mode(model,train_dataloader,loss_fn,accuracy_fn,optimizer)
    print(f"\nTrain loss: {train_loss:.5f} | Train accuracy: {train_accuracy:.5f}")
    test_loss, test_accuracy = test_mode(model,val_dataloader,loss_fn,accuracy_fn,optimizer)
    print(f"\nTrain loss: {test_loss:.5f} | Train accuracy: {test_accuracy:.5f}")

train_time_end_on_cpu = timer()
total_train_time_model = print_train_time(start=train_time_start_on_cpu, 
                                           end=train_time_end_on_cpu,
                                           device=str(next(model.parameters()).device))

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0
---------
Looked at 0/6090 samples

Train loss: 0.68659 | Train accuracy: 56.93227
Looked at 0/1523 samples

Train loss: 0.68541 | Train accuracy: 57.44565
Epoch: 1
---------
Looked at 0/6090 samples

Train loss: 0.68563 | Train accuracy: 57.05105
Looked at 0/1523 samples

Train loss: 0.68468 | Train accuracy: 57.40149
Epoch: 2
---------
Looked at 0/6090 samples

Train loss: 0.68512 | Train accuracy: 56.96791
Looked at 0/1523 samples

Train loss: 0.68401 | Train accuracy: 57.45301
Epoch: 3
---------
Looked at 0/6090 samples

Train loss: 0.68464 | Train accuracy: 56.96791
Looked at 0/1523 samples

Train loss: 0.68367 | Train accuracy: 57.37942
Epoch: 4
---------
Looked at 0/6090 samples

Train loss: 0.68433 | Train accuracy: 56.93227
Looked at 0/1523 samples

Train loss: 0.68330 | Train accuracy: 57.37942
Epoch: 5
---------
Looked at 0/6090 samples

Train loss: 0.68413 | Train accuracy: 56.88477
Looked at 0/1523 samples

Train loss: 0.68303 | Train accuracy: 57.37206
Epoch: 6
-